In [1]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Use your dataset path
dataset_path = r'C:\Users\lokes\Downloads\credit car fraud db\creditcard.csv'

# Load and prepare data
df = pd.read_csv(dataset_path)
X = df.drop(columns=["Class"])
y = df["Class"]
feature_names = X.columns.tolist()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

# Predict for a single transaction
def predict_single(*inputs):
    arr = np.array(inputs).reshape(1, -1)
    scaled = scaler.transform(arr)
    pred = model.predict(scaled)[0]
    prob = model.predict_proba(scaled)[0][1]
    return ("Fraud" if pred == 1 else "Legitimate", f"{prob*100:.2f}% fraud probability")

# Predict from uploaded CSV
def predict_file(file):
    data = pd.read_csv(file.name)
    if not set(feature_names).issubset(data.columns):
        return "CSV missing required features."
    scaled = scaler.transform(data[feature_names])
    preds = model.predict(scaled)
    probs = model.predict_proba(scaled)[:, 1]
    data["Prediction"] = np.where(preds == 1, "Fraud", "Legitimate")
    data["Fraud Probability (%)"] = (probs * 100).round(2)
    return data

# Build the Gradio UI
single_inputs = [gr.Number(label=f) for f in feature_names]

single_interface = gr.Interface(
    fn=predict_single,
    inputs=single_inputs,
    outputs=["text", "text"],
    title="Single Transaction Fraud Detection",
    description="Enter the 30 features manually to predict fraud."
)

batch_interface = gr.Interface(
    fn=predict_file,
    inputs=gr.File(label="Upload CSV File"),
    outputs=gr.Dataframe(),
    title="Batch Fraud Detection",
    description="Upload a CSV file with multiple transactions."
)

app = gr.TabbedInterface([single_interface, batch_interface], ["Single Entry", "Batch Upload"])
app.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
